In [46]:
import multiprocessing as mp
import pandas as pd
import numpy as np
import glob
from datetime import date, datetime, time, timedelta
from collections import defaultdict
import os

utilizando multiprocess, obtendre las temperaturas minimas de cada dia.
Primero, para evitar problemas, desplazare todos los dias en -20 hrs, de forma que cada dia empiece a las 20 hrs y termine a las 20 hrs del dia siguiente, asi obtendre la temperatura minima leida durante la noche (el dia es irrelevante)

In [5]:
path = "../data/cleared"
dfs = []
dict_meta = {"region": [], "estacion": [], "codigo": []}
for file in glob.glob(path + "/*/*"):
    print(file + "                                                                               ", end="\r")
    file = file.replace("\\", "/")
    dict_meta["region"].append(file.split("/")[-2])
    dict_meta["estacion"].append(file.split("/")[-1].split(".")[0])
    df = pd.read_csv(file)
    dict_meta["codigo"].append(df["Codigo"][0])
    dfs.append(df)
    

In [6]:
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['Codigo', 'Fecha', 'Hora'])
df = df.reset_index(drop=True)

In [8]:
df.head()

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica
0,13,2010-06-11,12:15:00,6.2,70.2,179.9,0.0,0.0,125.5,1008.0
1,13,2010-06-11,12:30:00,6.4,64.8,138.4,0.0,0.0,181.7,1007.0
2,13,2010-06-11,12:45:00,6.4,64.9,96.9,0.0,0.0,159.5,1007.0
3,13,2010-06-11,13:00:00,6.9,62.1,121.0,0.0,0.0,328.2,1008.0
4,13,2010-06-11,13:15:00,7.1,62.0,47.0,0.0,0.0,195.2,1007.0


In [11]:
# calculamos la nueva columna datetime y displaced_date
temp = np.array(df["Temperatura"])
code = np.array(df["Codigo"])
ddate = np.array(df["Fecha"])
dtime = np.array(df["Hora"])

ddtt = []
displaced_date = []
# create datetime object and displaceit by -19 hours
deltatime = timedelta(hours = 20)
c = 0
print(len(ddate))
for d, t in zip(ddate, dtime):
    
    if c % 10000 == 0:
        print(c, end="\r")
    if " " in d:
        d = d.split(" ")[0]
        datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
    elif "-" in d:
        datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
    else:
        datm = datetime.strptime(d + " " + t, '%d/%m/%Y %H:%M:%S')
    displaced_date.append((datm - deltatime).date().__str__())
    ddtt.append(datm)
    c += 1
    
    
# y la agregamos

df["datetime"] = ddtt
df["displaced_date"] = displaced_date

5301924


In [12]:
# botamos cualquier duplicado que exista, por ABC motivo
print(df.count())
df = df.drop_duplicates(subset=['Codigo', 'Fecha', 'Hora'], keep="first")
print(df.count())

Codigo                 5301924
Fecha                  5301924
Hora                   5301924
Temperatura            5301924
Humedad                5301924
Direccion de Viento    5301924
Velocidad de Viento    5301924
Precipitacion          5301924
Radiacion Solar        5301924
Presion Atmosferica    5301924
datetime               5301924
displaced_date         5301924
dtype: int64
Codigo                 5257742
Fecha                  5257742
Hora                   5257742
Temperatura            5257742
Humedad                5257742
Direccion de Viento    5257742
Velocidad de Viento    5257742
Precipitacion          5257742
Radiacion Solar        5257742
Presion Atmosferica    5257742
datetime               5257742
displaced_date         5257742
dtype: int64


In [15]:
# save this new dataframe
outdir = "../data/processed"
if not os.path.exists(outdir):
    os.makedirs(outdir)
df.to_csv(outdir + "/backup_dffull.csv", index=False, header=True)

## por precaucion, es recomendable resetear el notebook y cargar directamente el df creado, para limpiar memoria

In [64]:
outdir = "../data/processed"
df = pd.read_csv(outdir + "/backup_dffull.csv")
df = df.sort_values(by=['Codigo', 'displaced_date', 'Hora'])
df = df.reset_index(drop=True)
df

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica,datetime,displaced_date
0,13,2010-06-11,12:15:00,6.2,70.2,179.9,0.0,0.0,125.5,1008.0,2010-06-11 12:15:00,2010-06-10
1,13,2010-06-11,12:30:00,6.4,64.8,138.4,0.0,0.0,181.7,1007.0,2010-06-11 12:30:00,2010-06-10
2,13,2010-06-11,12:45:00,6.4,64.9,96.9,0.0,0.0,159.5,1007.0,2010-06-11 12:45:00,2010-06-10
3,13,2010-06-11,13:00:00,6.9,62.1,121.0,0.0,0.0,328.2,1008.0,2010-06-11 13:00:00,2010-06-10
4,13,2010-06-11,13:15:00,7.1,62.0,47.0,0.0,0.0,195.2,1007.0,2010-06-11 13:15:00,2010-06-10
5,13,2010-06-11,13:30:00,7.5,64.2,37.1,0.0,0.0,210.6,1006.0,2010-06-11 13:30:00,2010-06-10
6,13,2010-06-11,13:45:00,7.5,63.3,77.0,0.0,0.0,196.7,1006.0,2010-06-11 13:45:00,2010-06-10
7,13,2010-06-11,14:00:00,7.2,66.7,61.4,0.0,0.0,146.0,1006.0,2010-06-11 14:00:00,2010-06-10
8,13,2010-06-11,14:15:00,6.8,69.5,61.8,0.0,0.0,75.8,1006.0,2010-06-11 14:15:00,2010-06-10
9,13,2010-06-11,14:30:00,7.0,68.0,106.3,0.0,0.0,131.4,1006.0,2010-06-11 14:30:00,2010-06-10


In [69]:
# trabajaremos con la matrix para calcular las temperaturas minimas, es mas eficiente
data = df.values
print(df.columns)
type(data)

Index(['Codigo', 'Fecha', 'Hora', 'Temperatura', 'Humedad',
       'Direccion de Viento', 'Velocidad de Viento', 'Precipitacion',
       'Radiacion Solar', 'Presion Atmosferica', 'datetime', 'displaced_date'],
      dtype='object')


numpy.ndarray

agrupamos los datos en N subset, como diferentes estaciones pueden medir el mismo dia, separaremos por estaciones

In [49]:
# agrupamos los datos en N subset, tienen que ser dias completos, por lo que veamos primero cuantos dias son
total_codes = np.unique(df["Codigo"])
len(total_codes)

162

In [74]:
# uno puede elegir el tamaño de N, sera directamente la cantidad de workers que se lanzaran
N = 24
codes_per_worker = len(total_codes) // N
print(codes_per_worker)

6


In [75]:
ranges = [0]
c = 0
last_code = data[0][0]
i = 0
while i < df.shape[0]:
    if i % 1000 == 0:
        print(i, end="\r")
    di = data[i]
    if last_code != di[0]:
        c += 1
        last_code = di[0]
        if c == codes_per_worker:
            ranges.append(i)
            c = 0
    i += 1
ranges.append(data.shape[0])
ranges = np.array(ranges)
print(ranges.shape)
ranges

(28,)00


array([      0,   26678,   77475,  161990,  242822,  366412,  526204,
        749696, 1010546, 1286809, 1560976, 1836371, 2080444, 2352802,
       2641894, 2811515, 2987416, 3229789, 3517675, 3782765, 4015450,
       4205623, 4439147, 4610683, 4707174, 4913502, 5051518, 5257742])

In [78]:
# verifiquemos que este correcto
for i in range(len(ranges) - 1):
    print(len(np.unique(df.iloc[ranges[i]:ranges[i+1]]["Codigo"])), end=" ")

6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 

In [96]:
# ahora definimos el worker

def worker(data, ini, end, out_q):
    if True:
        results = defaultdict(list)
        print("processing range [{},{}]".format(ini, end))
        # primero obtenemos los sub indices de cada estacion (codigo)
        print("obteneindo sub indices ...", end="")
        sub_idxs = [ini]
        last_code = data[ini][0]
        for i in range(ini, end):
            if last_code != data[i][0]:
                last_code = data[i][0]
                sub_idxs.append(i)
        sub_idxs.append(end)
        print("done")
        
        # luego, recorremos estos rangos, identificamos el rango donde se encuentre las mediciones
        # para 1 solo dia y encontramos el idx donde ocurre la temperatura minima
        
        print("recorriendo datos ...")
        for i in range(len(sub_idxs) - 1):
            subini = sub_idxs[i]
            subend = sub_idxs[i+1]
            last_day = data[subini][11]
            subsubini = subini
            for j in range(subini, subend):
                dj = data[j]
                if last_day != data[j][11]:
                    last_day = data[j][11]
                    # obtenemos el arreglo de temperaturas
                    temps = data[subsubini:j,3]
                    idx_min = np.argmin(temps)
                    # y pasamos todo al diccionaro de salida
                    for k, name in zip([0, 1, 2, 10, 11, 3], 
                                   ["Codigo", "Fecha", "Hora", "datetime", "displaced_date", "minT"]):
                        results[name].append(data[subsubini + idx_min][k])
                    subsubini = j
        print("done")
                    
        out_q.put(results)
#     except:
#         print("worker failed")
#     finally:
#         print("done")


In [120]:
# probemos el worker en un pequeño rango de 1 codigo
idxs = df[df["Codigo"] == 450050].index
ini = idxs[0]
end = idxs[-1]
print(ini, end, end-ini)
m = mp.Manager()
result_queue = m.Queue()
worker(data, ini, end, result_queue)
x = result_queue.get()
pd.DataFrame(x).head()

5225570 5257741 32171
processing range [5225570,5257741]
obteneindo sub indices ...done
recorriendo datos ...
done


,Codigo,Fecha,Hora,datetime,displaced_date,minT
0,450050,2009-07-01,19:45:00,2009-07-01 19:45:00,2009-06-30,2.3
1,450050,2009-07-02,08:15:00,2009-07-02 08:15:00,2009-07-01,1.5
2,450050,2009-07-02,22:30:00,2009-07-02 22:30:00,2009-07-02,4.1
3,450050,2009-07-04,04:45:00,2009-07-04 04:45:00,2009-07-03,2.6
4,450050,2009-07-05,05:45:00,2009-07-05 05:45:00,2009-07-04,5.1


In [121]:
# otra prueba
ini = ranges[1]
end = ranges[2]
print(ini, end, end-ini)
m = mp.Manager()
result_queue = m.Queue()
worker(data, ini, end, result_queue)
x = result_queue.get()
pd.DataFrame(x).head()

26678 77475 50797
processing range [26678,77475]
obteneindo sub indices ...done
recorriendo datos ...
done


,Codigo,Fecha,Hora,datetime,displaced_date,minT
0,38,2010-04-29,07:15:00,2010-04-29 07:15:00,2010-04-28,14.5
1,38,2010-04-29,21:45:00,2010-04-29 21:45:00,2010-04-29,15.1
2,38,2010-05-01,03:15:00,2010-05-01 03:15:00,2010-04-30,17.6
3,38,2010-05-02,04:15:00,2010-05-02 04:15:00,2010-05-01,14.5
4,38,2010-05-03,07:00:00,2010-05-03 07:00:00,2010-05-02,12.6


programa final

In [122]:
# 
def run(data, ranges):
    m = mp.Manager()
    result_queue = m.Queue()
    jobs = []
    for i in range(len(ranges)-1):
        ini = ranges[i]
        end = ranges[i+1]
        jobs.append(mp.Process(target=worker, args=(data, ini, end, result_queue)))
        jobs[-1].start()
    
    for p in jobs:
        p.join()
    return result_queue

In [ ]:
# necesito testear esto
rqueue = run(data, ranges)